In [554]:
import pandas as pd
import folium
import json
import numpy as np
import requests

In [584]:
# df_trans=pd.read_csv('./연평균 지하철 이용객.csv')

In [586]:
# # kakao key 불러오기
# with open('C:/workspace/api_key/kakao_key.txt') as file:
#     kakao_key = file.read()

# # kakao_key를 통해서 모든 지하철역 주소 위도 경도 좌표 구하기
# url = "https://dapi.kakao.com/v2/local/search/keyword.json?"
# header = {"Authorization": "KakaoAK " + kakao_key}
# addr, lngs, lats = [], [], []
# for i in range(len(df_trans)):
#     query = df_trans['지하철역'][i]
#     if '역' not in df_trans['지하철역'][i]:
#         station = df_trans['지하철역'][i]
#         if '(' in df_trans['지하철역'][i]:
#             station = df_trans['지하철역'][i].split('(')[0]
#         query = station + '역'
#     result = requests.get(url, params = {'query':query}, headers={'Authorization' : 'KakaoAK ' + kakao_key}).json()['documents']
#     try:
#         addr.append(result[0]['address_name'])
#         lngs.append(float(result[0]['x']))
#         lats.append(float(result[0]['y']))
#     except:
#         print(df_trans['지하철역'][i])

In [587]:
# df_trans['주소'] = pd.DataFrame(addr)
# df_trans['위도'] = pd.DataFrame(lats)
# df_trans['경도'] = pd.DataFrame(lngs)

In [588]:
# df_trans_seoul = df_trans[df_trans['주소'].str.contains('서울')]

In [589]:
# df_trans_seoul.set_index('지하철역', inplace=True)

In [590]:
# df_trans_seoul.reset_index(inplace=True)

In [591]:
# df_trans_seoul.to_csv('seoul_stations.csv', index=False)

In [555]:
df_trans_seoul=pd.read_csv('./seoul_stations.csv')

In [592]:
df_trans_seoul.head(2)

,지하철역,호선명,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,07시-08시 하차인원,...,1시이후 승차,1시이후 하차,총 이용객,유임승차인원,무임승차인원,유임하차인원,무임하차인원,주소,위도,경도
0,가산디지털단지,1호선,1340.00,6.133333,8205.400000,1980.583333,7914.766667,10941.750000,14763.35,46802.533333,...,0.7,2.033333,9.784055e+05,418268.766667,29773.750000,497380.25,32982.716667,서울 금천구 가산동 468-4,37.480396,126.882662
1,개봉,1호선,1226.05,8.633333,18414.966667,5494.800000,36569.333333,11855.966667,95251.20,16530.883333,...,0.3,2.283333,1.322553e+06,514742.366667,145080.383333,511793.65,150936.250000,서울 구로구 개봉동 415,37.494642,126.858716


In [593]:
df_map=pd.concat([df_trans_seoul.iloc[:,:2],df_trans_seoul.iloc[:,-3:]], axis=1)

In [594]:
df_map

,지하철역,호선명,주소,위도,경도
0,가산디지털단지,1호선,서울 금천구 가산동 468-4,37.480396,126.882662
1,개봉,1호선,서울 구로구 개봉동 415,37.494642,126.858716
2,광운대,1호선,서울 노원구 월계동 85,37.623837,127.061695
3,구로,1호선,서울 구로구 구로동 585-3,37.503342,126.882308
4,구일,1호선,서울 구로구 구로동 636-45,37.496255,126.869691
...,...,...,...,...,...
372,응봉,중앙선,서울 성동구 응봉동 246,37.549953,127.034484
373,이촌(국립중앙박물관),중앙선,서울 용산구 이촌동 17-69,37.522413,126.973544
374,중랑,중앙선,서울 중랑구 중화동 73-7,37.594843,127.075680
375,청량리(서울시립대입구),중앙선,서울 동대문구 전농동 591-53,37.580733,127.048504


In [595]:
# 환승 가능한 역들
df_tran=[]
for i in df_trans_seoul['지하철역'].unique():
    index = df_trans_seoul[df_trans_seoul['지하철역'] == i].index
    if len(df_trans_seoul[df_trans_seoul['지하철역'] == i]) > 1:
        for j in index:
            df_tran.append(df_trans_seoul.iloc[j])
    else:
        pass

In [596]:
df_tran=pd.concat([pd.DataFrame(df_tran).iloc[:,:2],pd.DataFrame(df_tran).iloc[:,-3:]], axis=1)

In [597]:
df_tran=df_tran.groupby(['지하철역', '주소','위도','경도']).sum().reset_index()

In [598]:
df_tran

,지하철역,주소,위도,경도,호선명
0,가락시장,서울 송파구 가락동 184-23,37.493099,127.118263,3호선8호선
1,가산디지털단지,서울 금천구 가산동 468-4,37.480396,126.882662,1호선7호선
2,강남구청,서울 강남구 삼성동 111-44,37.517216,127.041311,7호선분당선
3,건대입구,서울 광진구 화양동 7-3,37.540408,127.069203,2호선7호선
4,고속터미널,서울 서초구 반포동 19-11,37.504549,127.005121,3호선7호선9호선
...,...,...,...,...,...
63,충정로(경기대입구),서울 서대문구 충정로3가 319-1,37.559763,126.964492,2호선5호선
64,태릉입구,서울 노원구 공릉동 616-4,37.618460,127.075405,6호선7호선
65,합정,서울 마포구 서교동 393,37.549913,126.914454,2호선6호선
66,홍대입구,서울 마포구 동교동 165,37.556871,126.923779,2호선경의선공항철도 1호선


In [599]:
#  01~09호선 분리
df_map_1=df_map[df_map['호선명']=='1호선']
df_map_2=df_map[df_map['호선명']=='2호선']
df_map_3=df_map[df_map['호선명']=='3호선']
df_map_4=df_map[df_map['호선명']=='4호선']
df_map_5=df_map[df_map['호선명']=='5호선']
df_map_6=df_map[df_map['호선명']=='6호선']
df_map_7=df_map[df_map['호선명']=='7호선']
df_map_8=df_map[df_map['호선명']=='8호선']
df_map_9=df_map[df_map['호선명']=='9호선']
df_map_10=df_map[df_map['호선명']=='경의선']
df_map_11=df_map[df_map['호선명']=='경춘선']
df_map_12=df_map[df_map['호선명']=='공항철도 1호선']
df_map_13=df_map[df_map['호선명']=='분당선']
df_map_14=df_map[df_map['호선명']=='우이신설선']
df_map_15=df_map[df_map['호선명']=='중앙선']

In [600]:
candidate=pd.read_csv('./환승역 후보.csv')

In [601]:
candidate

,지하철역,환승가능여부,출근시간대 승차,출근시간대 하차,퇴근시간대 승차,퇴근시간대 하차,유임이용객,무임이용객,총 이용객,유임이용객 비율,무임이용객 비율,면적,층수
0,학여울,N,11743.666667,18536.500000,18563.200000,11735.283333,1.157779e+05,28751.666667,1.445295e+05,0.801,0.199,28768.4,6
1,반포,N,29552.116667,51668.016667,48592.333333,30374.183333,2.903113e+05,59744.133333,3.500554e+05,0.829,0.171,12401.8,3
2,녹사평(용산구청),N,21001.133333,34967.350000,47723.966667,47658.150000,3.184402e+05,36007.016667,3.544472e+05,0.898,0.102,10677.6,5
3,월드컵경기장(성산),N,25702.850000,28018.416667,45758.116667,41734.216667,2.656673e+05,109136.500000,3.748038e+05,0.709,0.291,12286.2,3
4,서울숲,N,51952.283333,62128.166667,75630.316667,52836.433333,4.411345e+05,46166.383333,4.873009e+05,0.905,0.095,20369.0,4
5,언주,N,22171.966667,97975.600000,97481.450000,35592.850000,4.454473e+05,46779.466667,4.922268e+05,0.905,0.095,11552.8,5
6,봉화산(서울의료원),N,113918.250000,25789.866667,31708.766667,80241.983333,4.146879e+05,94950.266667,5.096382e+05,0.814,0.186,20369.0,4
7,일원,N,59327.933333,67197.550000,54308.916667,50060.300000,4.129607e+05,111298.350000,5.242591e+05,0.788,0.212,10573.0,3
8,영등포시장,N,52103.700000,70814.866667,77604.600000,57533.450000,4.067348e+05,135583.166667,5.423179e+05,0.750,0.250,13407.2,6
9,여의나루,N,19209.816667,106458.083333,122446.900000,72300.366667,5.960562e+05,53620.650000,6.496769e+05,0.917,0.083,10598.9,5


In [602]:
candidate_location=[]
for i in range(0,len(candidate['지하철역'])):
    candidate_location=pd.concat([pd.DataFrame(candidate_location),df_map[df_map['지하철역']==candidate['지하철역'][i]]], axis=0)

In [603]:
candidate_location=candidate_location.reset_index()
candidate_location.drop(columns='index', inplace=True)

In [604]:
candidate_location

,지하철역,호선명,주소,위도,경도
0,학여울,3호선,서울 강남구 대치동 514-3,37.496885,127.071167
1,반포,7호선,서울 서초구 잠원동 103,37.508161,127.011586
2,녹사평(용산구청),6호선,서울 용산구 용산동4가 4,37.534723,126.986550
3,월드컵경기장(성산),6호선,서울 마포구 성산동 420,37.569636,126.899098
4,서울숲,분당선,서울 성동구 성수동1가 656-436,37.543646,127.044746
5,언주,9호선,서울 강남구 논현동 279-165,37.507335,127.033971
6,봉화산(서울의료원),6호선,서울 중랑구 신내동 643-1,37.617351,127.091360
7,일원,3호선,서울 강남구 일원동 717,37.483989,127.084130
8,영등포시장,5호선,서울 영등포구 영등포동5가 62-1,37.522709,126.905180
9,여의나루,5호선,서울 영등포구 여의도동 84-4,37.527147,126.932986


In [569]:
# # 환승 후보역 지도에 표시
# for i in candidate_location.index: 
#     folium.Marker(
#         radius=200, 
#         location=[candidate_location.위도[i],candidate_location.경도[i]],
#         popup=folium.Popup(candidate_location.호선명[i], max_width=200),
#         tooltip=candidate_location.지하철역[i]+'(환승후보역)',
#         color='yellow',
#         icon = folium.Icon(
#         icon='train-subway')
#     ).add_to(map)

In [623]:
map=folium.Map(location=[df_map.위도.mean(),df_map.경도.mean()], zoom_start=11,
               tiles='Stamen Terrain') 

r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

map.choropleth(geo_data=seoul_geo,
             line_color='black',
             fill_opacity=0.1,
             line_opacity=1,
             line_weight=2
            )

colors=['darkblue','darkgreen','darkorange','darkturquoise','slateblue','chocolate','olive',
      'deeppink','tan','mediumaquamarine','forestgreen','dodgeblue','gold','yellogreen','mediumaquamarine']


# 환승 후보역 지도에 표시
for i in candidate_location.index: 
    folium.Marker(
        radius=1000, 
        location=[candidate_location.위도[i],candidate_location.경도[i]],
        popup=folium.Popup(candidate_location.호선명[i], max_width=200),
        tooltip=candidate_location.지하철역[i]+'(환승후보역)',
        icon = folium.Icon(
            color='purple',
            icon='train-subway',
            prefix='fa'
        )
    ).add_to(map)
for i in candidate_location.index: 
    folium.Circle(
        radius=2000, 
        location=[candidate_location.위도[i],candidate_location.경도[i]],
        popup=folium.Popup(candidate_location.호선명[i], max_width=200),
        tooltip=candidate_location.지하철역[i]+'(환승후보역)',
        color='red',
        fill=True,
        fill_opacity=0.2,
        fill_color='red'
    ).add_to(map)

for i in range(1,16):
    for j in globals()[f'df_map_{i}'].index:
        folium.Circle(
        radius=200, 
        location=[globals()[f'df_map_{i}'].위도[j],globals()[f'df_map_{i}'].경도[j]],
        popup=folium.Popup(globals()[f'df_map_{i}'].호선명[j], max_width=200),
        tooltip=globals()[f'df_map_{i}'].지하철역[j],
        color=colors[i-1],
        fill=True,
        fill_color=colors[i-1]
    ).add_to(map)

for i in df_tran.index: 
    folium.Circle(
        radius=200, 
        location=[df_tran.위도[i],df_tran.경도[i]],
        popup=folium.Popup(df_tran.호선명[i], max_width=200),
        tooltip=df_tran.지하철역[i]+'(환승가능역)',
        color='red',
        fill=True,
        fill_opacity=1,
        fill_color='red'
    ).add_to(map)
    

    
title='<h3 align="center" style="font-size:20px">서울시 지하철 노선</h3>'
map.get_root().html.add_child(folium.Element(title))

map

C:\Users\user\anaconda3\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [571]:
from geopy import distance

In [606]:
df_map=df_map.groupby(['지하철역', '주소','위도','경도']).sum().reset_index()

In [607]:
df_map.loc[df_map[df_map['지하철역'] == '용산'].index, '호선명'] = '1호선중앙선'

In [608]:
df_map[df_map['지하철역'] == '용산']

,지하철역,주소,위도,경도,호선명
230,용산,서울 용산구 한강로3가 40-999,37.530076,126.964824,1호선중앙선


In [609]:
def creat_cand(can_gps, df_map, km):
    cand = []
    # 후보역 반경 2km이내의 역 추출
    for i in df_map.index:
        map_gps = df_map.loc[i, '위도'],df_map.loc[i, '경도']
        dist = distance.distance(can_gps, map_gps).km
        if (dist < km) & (dist > 0):
            cand.append(df_map.loc[i])
    return pd.DataFrame(cand)

In [610]:
def store_line(lines, line):
    if len(line) > 0:
        line = line.split('선')[0:-1]
        for li in line:
            li = li + '선'
            if li not in lines:
                lines.append(li)
    return lines

In [611]:
def store_lines(lines, candidate_station, cand):
    for i in cand.index:
        # 같은호선인 환승역들의 환승노선 저장
        if candidate_station['호선명'] in cand.loc[i, '호선명']:
            line = cand.loc[i, '호선명'].replace(candidate_station['호선명'], '')
            lines = store_line(lines, line)

        # 제거할 다른 노선 환승역의 환승노선 저장
        else:
            for li in lines:
                if li in cand.loc[i, '호선명']:
                    line = cand.loc[i, '호선명'].replace(li, '')
                    lines = store_line(lines, line)
    return lines

In [612]:
def remove_cand(cand, lines):
    for i in cand.index:
        if cand.loc[i, '호선명'].count('선') > 1:
            cand.drop(index=i, inplace=True)
        else:
            for line in lines:
                if line in cand.loc[i, '호선명']:
                    cand.drop(index=i, inplace=True)
                    break
    return cand

In [639]:
candidate_location_copy = candidate_location.copy()

In [640]:
choice = pd.DataFrame(columns=['지하철역', '주소', '위도', '경도', '호선명'])
for i in candidate_location.index:
    km = 2
    can_gps = candidate_location.loc[i, '위도'],candidate_location.loc[i, '경도']
    cand = creat_cand(can_gps, df_map, km)
    lines = [candidate_location.loc[i]['호선명']]
    candidate_station = candidate_location.loc[i]
    for j in range(3):
        lines = store_lines(lines, candidate_station, cand)
    cand = remove_cand(cand, lines)
    
    if len(cand) == 0:
        candidate_location_copy.drop(index=i, inplace=True)
    else:
        choice = pd.concat([choice, cand])

In [641]:
choice

,지하철역,주소,위도,경도,호선명
132,삼성(무역센터),서울 강남구 삼성동 159-8,37.509192,127.060804,2호선
136,삼전,서울 송파구 잠실동 347,37.504549,127.087364,9호선
89,뚝섬,서울 성동구 성수동1가 656-745,37.547242,127.047387,2호선
159,성수,서울 성동구 성수동2가 300-1,37.544589,127.056067,2호선
238,응봉,서울 성동구 응봉동 246,37.549953,127.034484,중앙선
282,한양대,서울 성동구 행당동 산 17,37.555716,127.043640,2호선
186,신사,서울 강남구 신사동 667,37.516436,127.020309,3호선
86,둔촌오륜,서울 강동구 둔촌동 227-7,37.519408,127.138652,9호선
261,중앙보훈병원,서울 강동구 둔촌동 8-1,37.528403,127.148412,9호선
6,강남,서울 강남구 역삼동 858,37.498086,127.028001,2호선


In [642]:
candidate_location_copy

,지하철역,호선명,주소,위도,경도
0,학여울,3호선,서울 강남구 대치동 514-3,37.496885,127.071167
4,서울숲,분당선,서울 성동구 성수동1가 656-436,37.543646,127.044746
5,언주,9호선,서울 강남구 논현동 279-165,37.507335,127.033971
13,강동,5호선,서울 강동구 천호동 447,37.535882,127.132396
14,논현,7호선,서울 강남구 논현동 279-67,37.511200,127.021654
16,경복궁(정부서울청사),3호선,서울 종로구 적선동 81-1,37.575806,126.973691
21,강변(동서울터미널),2호선,서울 광진구 구의동 546-6,37.535118,127.094741
